In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#import seaborn as sns
#%matplotlib inline
import re
import time
from datetime import datetime
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests

In [22]:
#Quantidade maxima de páginas a serem extraidas
no_pages = 2

#Termo de busca digitado pelo user
search = "dan+brown"

def get_data(pageNo, searchTitle):
    #Configuracao do BeatifulSoup  
    headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}
    
    # MODELO DE URL DO SITE - https://www.estantevirtual.com.br/busca?utf8=%E2%9C%93&q=dan%20brown&livro_novo=1&offset=2
    r = requests.get('https://www.estantevirtual.com.br/busca?q='+str(searchTitle)+'&livro_novo=1&offset='+str(pageNo), headers=headers)#, proxies=proxies)
    content = r.content
    soup = BeautifulSoup(content)
    #print(soup)

    alls = []

    #Encontra no html todas as divs com essa class e pega seu conteúdo
    for d in soup.findAll('div', attrs={'class':'livro exibe-desagrupado'}):
        #print(d)
        name = d.find('h2', attrs={'itemprop':'name'})
        name = name.text
        name = re.sub(r'\n|\n ', '', name)

        #n = name.find_all('img', alt=True)
        #print(n[0]['alt'])
        author = d.find('span', attrs={'itemprop':'author'} )
        author = author.text
        author = re.sub(r'\n|\n ', '', author)
        #rating = d.find('span', attrs={'class':'4,1 de 5 estrelas'})
        #r = rating.find_all('span', aria-label=True)
        
        price = d.find('span', attrs={'class':'preco'})

        imgUrl = d.find('div', attrs={'class': 'capa'}).find('img')['data-src']
        
        siteUrl = d.find('a', attrs={'class': 'livro-link js-ab-nvbbx'})['href']

        all1=[]

        if name is not None:
            #print(n[0]['alt'])
            all1.append(name)
        else:
            all1.append("unknown-product")

        if author is not None:
            #print(author.text)
            all1.append(author)
        elif author is None:
            author = d.find('span', attrs={'class':'a-size-small a-color-base'})
            if author is not None:
                all1.append(author.text)
            else:    
                all1.append('0')
        
        if imgUrl is not None:
            all1.append(imgUrl)
        else:
            all1.append("invalid-image")
        
        if siteUrl is not None:
            all1.append(siteUrl)
        else:
            all1.append("invalid-url-site")

        #if rating is not None:
            #print(rating.text)
        #    all1.append(rating.text)
        #else:
        #    all1.append('-1')

        #if users_rated is not None:
            #print(price.text)
        #    all1.append(users_rated.text)
        #else:
        #    all1.append('0')     

        if price is not None:
            #print(price.text)
            all1.append(price.text)
        else:
            all1.append('0')
        alls.append(all1)    
    return alls

In [23]:
results = []
for i in range(1, no_pages+1):
    results.append(get_data(i, search))
flatten = lambda l: [item for sublist in l for item in sublist]
df = pd.DataFrame(flatten(results),columns=['Titulo','Autor', 'ImgUrl', 'SiteUrl', 'Preco'])
df.to_csv('estante_products.csv', index=False, encoding='utf-8')

In [24]:
df = pd.read_csv("estante_products.csv")
df.shape

(42, 5)

In [25]:
df.head(61)

,Titulo,Autor,ImgUrl,SiteUrl,Preco
0,O Caderno de Viagens de o Codigo da Vinc...,Baseado no Livro de Dan Brown,https://d1pkzhm5uq4mnt.cloudfront.net/imagens/...,https://www.estantevirtual.com.br/sylviajones/...,"R$ 4,00"
1,Desvendando o Inferno,Dan Brown,https://d1pkzhm5uq4mnt.cloudfront.net/imagens/...,https://www.estantevirtual.com.br/sebobooksonl...,"R$ 9,90"
2,Ponto de Impacto P,Dan Brown,https://d1pkzhm5uq4mnt.cloudfront.net/imagens/...,https://www.estantevirtual.com.br/cia-do-livro...,"R$ 10,00"
3,O Código da Vinci P,Dan Brown,https://d1pkzhm5uq4mnt.cloudfront.net/imagens/...,https://www.estantevirtual.com.br/cia-do-livro...,"R$ 12,00"
4,O Código da Vinci P,Dan Brown,https://d1pkzhm5uq4mnt.cloudfront.net/imagens/...,https://www.estantevirtual.com.br/cia-do-livro...,"R$ 13,00"
5,O Caderno de Viagens de o Codigo da Vinc...,Dan Brown,https://d1pkzhm5uq4mnt.cloudfront.net/imagens/...,https://www.estantevirtual.com.br/luklivros/da...,"R$ 14,00"
6,O Caderno de Viagens de o Codigo da Vinc...,Dan Brown,https://d1pkzhm5uq4mnt.cloudfront.net/imagens/...,https://www.estantevirtual.com.br/luklivros/da...,"R$ 14,00"
7,O Caderno de Viagens de o Codigo da Vinc...,Dan Brown,https://d1pkzhm5uq4mnt.cloudfront.net/imagens/...,https://www.estantevirtual.com.br/luklivros/da...,"R$ 14,00"
8,O Caderno de Viagens de o Codigo da Vinc...,Dan Brown,https://d1pkzhm5uq4mnt.cloudfront.net/imagens/...,https://www.estantevirtual.com.br/luklivros/da...,"R$ 14,00"
9,O Caderno de Viagens de o Codigo da Vinc...,Dan Brown,https://d1pkzhm5uq4mnt.cloudfront.net/imagens/...,https://www.estantevirtual.com.br/luklivros/da...,"R$ 14,00"
